In [1]:
# Large farm problem to test scalability of PIPS-NLP
# Yankai Cao and Victor M. Zavala
# University of Wisconsin-Madison, 2016

using JuMP
using Distributions
using Ipopt
using Plasmo
 # Initialize MPI
MPI.Init()

srand(123)
NS = 100;                   # number of scenarios
NP = 100;                   # number of products
S = collect(1:NS)           # scenario set
P = collect(1:NP)           # set of crops (1=wheat,2=corn,3=beets)

prcost = zeros(NP)
d = Uniform(100,200)
prcost = rand(d,NP)

pcost = zeros(NP)
d = Uniform(100,200)
pcost = rand(d,NP)

scost = zeros(NP)
scost = pcost - 50

demand = zeros(NP)
d = Uniform(100,300)
demand = rand(d,NP)/NP

# assign random data
yield = zeros(length(S),NP)
d = Uniform(5,20)
for j in 1:(NP-1)
    yield[S,j] = rand(d,1)[1]
end
d = Uniform(10,30)
yield[S,NP] = rand(d,NS)

sellub = zeros(NP)
d = Uniform(2000,8000)
sellub[P] = rand(d,NP)

# create PLASMO model and solve with PIPS-NLP
graph = GraphModel()
master = Model()
master_node = add_node(graph,master)

@variable(master, x[P] >= 0)
@variable(master, s2 >= 0)
@constraint(master, cap, (sum(x[j] for j in P) + s2) == 200)
@objective(master, Min, sum(prcost[j]*x[j] for j in P))
child_nodes = Array{Plasmo.NodeOrEdge}(NS)
for i in 1:NS
    bl = Model()
    child_node = add_node(graph,bl)
    child_nodes[i] = child_node
    @variable(bl, y[P] >= 0)    # crops purchase
    @variable(bl, 0<=w[j in P] <= sellub[j in P])    # crops sold
    @variable(bl, s[P] >= 0)
    @linkconstraint(graph, [j in P], yield[i,j]*master[:x][j]+y[j]-w[j] - s[j] == demand[j])
    @objective(bl, Min, 1.0/NS*sum(pcost[j]*y[j] - scost[j]*w[j] for j in P))
end

# solve with PipsNLP
pipsnlp_solve(graph,master_node,child_nodes)

#println(getvalue(x))
#for i in 1:NS
#    println(getvalue(getindex(child_nodes[i],:w))) 
#end

MPI.Finalize()


load option file: pipsnlp.parameter 
OPTION: Set DoIR_Aug to 1
OPTION: Set DoIR_Full to 0
OPTION: Set printing level to 2
OPTION: Set Iteration Limit to 500
OPTION: Set Convergence Tolerance to 1.00e-06
OPTION: Max Line search step:   50
OPTION: Set Sym linear solver as MA57.
OPTION: MA57 pivot level = 1.00e-04
OPTION: MA57 ordering method = 5, (see doc)
OPTION: Do IR on Aug sys.
OPTION: Set MaxIR to 10
OPTION: Set IRtol to 1.00e-12
OPTION: Require small constraint violation in SWC 
  
  -----------------------------------------------
  NLP Solver 
  Argonne National Laboratory, 2016
  -----------------------------------------------


  Linear system solver ------	 Ma57.

solving ...
1st stage 101 variables, 1 equality constraints, 0 inequality constraints.
2nd stage (use 1st scenario): 300 variables, 100 equality constraints, 0 inequality constraints.
100 scenarios.
Total 30101 variables, 10001 equality constraints, 0 inequality constraints. 

 Iter	  Objective     Inf_pr	  Inf_du    

init_x0  0.0
str_init_x0  0.0
eval_f  0.07422235000000023
eval_g0  0.2903103510000001
eval_grad_f  0.06267655800000006
eval_jac  0.8324692860000074
str_eval_jac  0.0
eval_h  0.0
str_eval_h 0.31704646100000206
eval_write_solution  0.010150565
Solution time:   41.008998963 (s)
